In [ ]:
import pandas as pd
import numpy as np
import datetime

#Definindo a opção para exibir todas as colunas com 2 casas decimais
pd.set_option('display.float_format', '{:.2f}'.format)

###################################################################################################
#############  Configurações de Período de Análise   #############
# OBS: Análise de Janeiro, colocar o Mês Anterior 13 (Encerramento do Ano Anterior) >> e usar ´´Caminho`` próprio para a PASTA
mes = '09'
mes_anterior = '08'
ano = '2025'


######################################################################

msc_base = pd.read_excel(caminho_msc, header=1)
deta_ant = pd.read_excel(caminho_deta_ant, header=3)
deta = pd.read_excel(caminho_deta, header=3)
rec = pd.read_excel(caminho_rec, header=3, dtype=object)
dps = pd.read_excel(caminho_dps, header=3, dtype=object)
rp = pd.read_excel(caminho_rp, header=3, dtype=object)


###################################    ITEM 1 - POR GRUPO DE CONTAS COMPARANDO MSC do mes  e FLEX do mes Anterior  ########################################################################################

# Importando os Arquivos
msc = msc_base.copy()

###################################### RESUMINDO A MSC
# Transfomar a coluna CONTA em String
msc['CONTA'] = msc['CONTA'].apply(str)

# Pegar só o primeiro elemento de uma coluna
msc["Grupo de Contas"] = msc["CONTA"].str[0]

# trocar o sinal
msc['VALOR'] = msc.apply(lambda x: x['VALOR'] * -1
if (x['Grupo de Contas'] == '1' and x['NATUREZA_VALOR'] == 'C' and not x['TIPO_VALOR'] == 'period_change')
or (x['Grupo de Contas'] == '2' and x['NATUREZA_VALOR'] == 'D' and not x['TIPO_VALOR'] == 'period_change')
or (x['Grupo de Contas'] == '4' and x['NATUREZA_VALOR'] == 'D' and not x['TIPO_VALOR'] == 'period_change')
or (x['Grupo de Contas'] == '5' and x['NATUREZA_VALOR'] == 'C' and not x['TIPO_VALOR'] == 'period_change')
or (x['Grupo de Contas'] == '6' and x['NATUREZA_VALOR'] == 'D' and not x['TIPO_VALOR'] == 'period_change')
or (x['Grupo de Contas'] == '7' and x['NATUREZA_VALOR'] == 'C' and not x['TIPO_VALOR'] == 'period_change')
or (x['Grupo de Contas'] == '8' and x['NATUREZA_VALOR'] == 'D' and not x['TIPO_VALOR'] == 'period_change')
else x['VALOR'], axis=1)

# Resumindo
msc = msc.groupby(['Grupo de Contas', 'TIPO_VALOR'])['VALOR'].sum().reset_index()
msc = msc.loc[msc['TIPO_VALOR'] == 'beginning_balance']
msc = msc.filter(items=['Grupo de Contas', 'VALOR'])
# Resetando o INDEX e criando um novo DF
msc = msc.set_index("Grupo de Contas")

##########################################################################
##########################################################################

# Deletando a primeira Linha
deta_ant.drop(0, axis=0, inplace=True)
# Deletando a parte final do arquivo
deta_ant.drop(deta_ant.tail(17).index, inplace=True)
# Transformando a Coluna em String
deta_ant['Contas'] = deta_ant['Contas'].apply(str)
# Criando o Grupo de Contas
deta_ant["Grupo de Contas"] = deta_ant["Contas"].str[0]
#Somando as Colunas
deta_ant['period_change'] = deta_ant['Movimento'] + deta_ant['Unnamed: 3']

# Deletando Colunas
deta_ant.drop('Movimento', axis=1, inplace=True)
deta_ant.drop('Contas', axis=1, inplace=True)
deta_ant.drop('Unnamed: 3', axis=1, inplace=True)

# Renomeando as Colunas
deta_ant.columns = ['beginning_balance', 'ending_balance', 'Grupo de Contas', 'period_change']
# Selecionando Colunas Pelos Nomes
deta_ant = deta_ant.filter(items=['Grupo de Contas', 'ending_balance'])
deta_ant.rename(columns={"ending_balance": "VALOR"}, inplace=True)

# Resetando o index
deta_ant = deta_ant.set_index("Grupo de Contas")
deta_ant['VALOR'] = pd.to_numeric(deta_ant['VALOR'],errors = 'coerce')

############################# Criando um Arquivo FINAL

# Criando o Merge entre os DF
final_ant = msc.merge(deta_ant, on='Grupo de Contas')
# Resetando o indice
final_ant = final_ant.reset_index()
#Alterando a Coluna Grupo de Contas
gc = {'1': 'Ativo_1', '2': 'Passivo_2', '3': 'VPD_3', '4': 'VPA_4', '5': 'Controle_5', '6': 'Controle_6', '7': 'Controle_7', '8': 'Controle_8'}
# Substituir os códigos pelos nomes
final_ant['Grupo de Contas'] = final_ant['Grupo de Contas'].replace(gc)
# Fazendo a diferença
final_ant['DIF'] = final_ant['VALOR_x'] - final_ant['VALOR_y']
# Renomeando Colunas
final_ant.columns = ['Grupo de Contas', 'SI_MSC_mes atual','SF_FLEX_mes anterior','DIF']

# Convertendo as colunas para tipo numérico e tratando NaNs
final_ant['DIF'] = pd.to_numeric(final_ant['DIF'], errors='coerce').fillna(0)

# Supondo que tolerancia seja um valor definido anteriormente no seu código
tolerancia = 0.01  # Exemplo de tolerância
# Sua condição ajustada usando numpy.isclose para verificar a proximidade com zero
condicao1 = ~np.isclose(final_ant['DIF'], 0, atol=tolerancia)

if condicao1.any():
    resposta_final_ant= 'ERRO'
else:
    resposta_final_ant = 'OK'

final_ant_a = pd.DataFrame([resposta_final_ant], columns=['Resposta'])
# Insere novas colunas
final_ant_a.insert(0, 'Análise', 'Item 1')
final_ant_a.insert(2, 'Descrição da Análise', 'Análise do Saldo Final do Mês Anterior com o Saldo Inicial da MSC')



###############################################  ITEM 2 - ANÁLISE POR GRUPO DE CONTAS  ############################################################################
msc = msc_base.copy()    
###################################### RESUMINDO A MSC
# Transfomar a coluna CONTA em String
msc['CONTA'] = msc['CONTA'].apply(str)
# Pegar só o primeiro elemento de uma coluna
msc["Grupo de Contas"] = msc["CONTA"].str[0]

# Aplicando a fórmula para trocar o sinal das contas retificadoras
msc['VALOR'] = msc.apply(lambda x: x['VALOR'] * -1
if (x['Grupo de Contas'] == '1' and x['NATUREZA_VALOR'] == 'C' and not x['TIPO_VALOR'] == 'period_change')
or (x['Grupo de Contas'] == '2' and x['NATUREZA_VALOR'] == 'D' and not x['TIPO_VALOR'] == 'period_change')
or (x['Grupo de Contas'] == '4' and x['NATUREZA_VALOR'] == 'D' and not x['TIPO_VALOR'] == 'period_change')
or (x['Grupo de Contas'] == '5' and x['NATUREZA_VALOR'] == 'C' and not x['TIPO_VALOR'] == 'period_change')
or (x['Grupo de Contas'] == '6' and x['NATUREZA_VALOR'] == 'D' and not x['TIPO_VALOR'] == 'period_change')
or (x['Grupo de Contas'] == '7' and x['NATUREZA_VALOR'] == 'C' and not x['TIPO_VALOR'] == 'period_change')
or (x['Grupo de Contas'] == '8' and x['NATUREZA_VALOR'] == 'D' and not x['TIPO_VALOR'] == 'period_change')
else x['VALOR'], axis=1)

# Aplicando a fórmula para trocar o sinal do period_change
msc['VALOR'] = msc.apply(lambda x: x['VALOR'] * -1
if (x['Grupo de Contas'] == '1' and x['NATUREZA_VALOR'] == 'C' and x['TIPO_VALOR'] == 'period_change')
or (x['Grupo de Contas'] == '2' and x['NATUREZA_VALOR'] == 'D' and x['TIPO_VALOR'] == 'period_change')
or (x['Grupo de Contas'] == '3' and x['NATUREZA_VALOR'] == 'C' and x['TIPO_VALOR'] == 'period_change')
or (x['Grupo de Contas'] == '4' and x['NATUREZA_VALOR'] == 'D' and x['TIPO_VALOR'] == 'period_change')
or (x['Grupo de Contas'] == '5' and x['NATUREZA_VALOR'] == 'C' and x['TIPO_VALOR'] == 'period_change')
or (x['Grupo de Contas'] == '6' and x['NATUREZA_VALOR'] == 'D' and x['TIPO_VALOR'] == 'period_change')
or (x['Grupo de Contas'] == '7' and x['NATUREZA_VALOR'] == 'C' and x['TIPO_VALOR'] == 'period_change')
or (x['Grupo de Contas'] == '8' and x['NATUREZA_VALOR'] == 'D' and x['TIPO_VALOR'] == 'period_change')
else x['VALOR'], axis=1)

# Resumindo 
msc = msc.groupby(['Grupo de Contas', 'TIPO_VALOR'])['VALOR'].sum().reset_index()

# Pivoteando pelo Tipo_Valor
msc = msc.pivot_table(index=['Grupo de Contas'],
        columns='TIPO_VALOR',
        values='VALOR').reset_index()

# Resetando o INDEX e criando um novo DF
msc = msc.set_index("Grupo de Contas")

#######################################################
#######################################################
#######################################################
#######################################################
# Deletando a primeira Linha
deta.drop(0, axis=0, inplace=True)
# Deletando a parte final do arquivo
deta.drop(deta.tail(17).index, inplace=True)
# Transformando a Coluna em String
deta['Contas'] = deta['Contas'].apply(str)
# Criando o Grupo de Contas
deta["Grupo de Contas"] = deta["Contas"].str[0]
# Aplicando a fórmula para trocar o sinal do Movimento do período (substituir os valores por vezes -1)
deta.loc[deta['Grupo de Contas'] == "1", 'Unnamed: 3'] *= -1
deta.loc[deta['Grupo de Contas'] == "2", 'Movimento'] *= -1
deta.loc[deta['Grupo de Contas'] == "3", 'Unnamed: 3'] *= -1
deta.loc[deta['Grupo de Contas'] == "4", 'Movimento'] *= -1
deta.loc[deta['Grupo de Contas'] == "5", 'Unnamed: 3'] *= -1
deta.loc[deta['Grupo de Contas'] == "6", 'Movimento'] *= -1
deta.loc[deta['Grupo de Contas'] == "7", 'Unnamed: 3'] *= -1
deta.loc[deta['Grupo de Contas'] == "8", 'Movimento'] *= -1

#Somando as Colunas
deta['period_change'] = deta['Movimento'] + deta['Unnamed: 3']
deta = deta.filter(items=['Grupo de Contas', 'Saldo Anterior', 'Saldo', 'period_change'])
deta.columns = ['Grupo de Contas', 'beginning_balance', 'ending_balance', 'period_change']
deta = deta.set_index("Grupo de Contas")

#######################################################
#######################################################
#######################################################

# Concatenar os DataFrames lado a lado
final = pd.concat([msc, deta], axis=1)
final = final.reset_index()
#Alterando a Coluna Grupo de Contas
gc = {'1': 'Ativo_1', '2': 'Passivo_2', '3': 'VPD_3', '4': 'VPA_4', '5': 'Controle_5', '6': 'Controle_6', '7': 'Controle_7', '8': 'Controle_8'}
# Substituir os códigos pelos nomes
final['Grupo de Contas'] = final['Grupo de Contas'].replace(gc)
# Renomeando Colunas
final.columns = ['Grupo de Contas', 'SI_MSC', 'SF_MSC', 'Mov_MSC',
            'SI_FLEX', 'SF_FLEX', 'Mov_FLEX']

final['SI_FLEX'] = pd.to_numeric(final['SI_FLEX'],errors = 'coerce')
final['SF_FLEX'] = pd.to_numeric(final['SF_FLEX'],errors = 'coerce')
final['Mov_FLEX'] = pd.to_numeric(final['Mov_FLEX'],errors = 'coerce')

final['DIF_SI'] = final['SI_MSC'] - final['SI_FLEX']
final['DIF_SF'] = final['SF_MSC'] - final['SF_FLEX']
final['DIF_Mov'] = final['Mov_MSC'] - final['Mov_FLEX']

#Apagando a última linha com o drop
final = final.drop(final.index[-1])


# Supondo que tolerancia seja um valor definido anteriormente no seu código
tolerancia = 0.01  # Exemplo de tolerância
# Sua condição ajustada usando numpy.isclose para verificar a proximidade com zero
condicao1 = ~np.isclose(final['DIF_SI'], 0, atol=tolerancia)
condicao2 = ~np.isclose(final['DIF_SF'], 0, atol=tolerancia)
condicao3 = ~np.isclose(final['DIF_Mov'], 0, atol=tolerancia)

if condicao1.any() or condicao2.any() or condicao3.any():
    resposta_final = 'ERRO'
else:
    resposta_final = 'OK'

final_a = pd.DataFrame([resposta_final], columns=['Resposta'])
# Insere novas colunas
final_a.insert(0, 'Análise', 'Item 2')
final_a.insert(2, 'Descrição da Análise', 'Análise do Saldo Total por Grupo de Contas')



###############################################  ITEM 3 - REC REALIZADA  ############################################################################

############################# RESUMINDO A MSC ref. a REC REALIZADA

msc_rec = msc_base.copy()
#Transfomar as colunas em String
msc_rec['CONTA'] = msc_rec['CONTA'].apply(str)
msc_rec['IC1'] = msc_rec['IC1'].apply(str)
msc_rec['IC2'] = msc_rec['IC2'].apply(str)
msc_rec['IC3'] = msc_rec['IC3'].apply(str)
msc_rec['IC4'] = msc_rec['IC4'].apply(str)
msc_rec['IC5'] = msc_rec['IC5'].apply(str)
msc_rec['IC6'] = msc_rec['IC6'].apply(str)

#Pegando a Receita Realizada
msc_rec = msc_rec[msc_rec['CONTA'].str.contains("6212", case=False, regex=True)]
#Agrupando
#msc_rec = msc_rec.groupby(['IC1', 'IC2', 'TIPO_VALOR', 'NATUREZA_VALOR'])['VALOR'].sum().reset_index()
msc_rec = msc_rec.groupby(['IC1', 'IC2', 'TIPO_VALOR'])['VALOR'].sum().reset_index()
#Pivoteando o TIPO_VALOR para colunas
msc_rec = msc_rec.pivot_table(index=['IC1', 'IC2'],
            columns='TIPO_VALOR',
            values='VALOR').reset_index()

#Filtrando só as Colunas
msc_rec = msc_rec.filter(items=['IC1', 'IC2', 'beginning_balance', 'ending_balance'])
#Renomeando as Colunas
msc_rec.columns = ['PODER', 'FONTE_STN', 'SI_MSC', 'SF_MSC']
#Somando as Colunas
msc_rec['PODER_FONTE'] = msc_rec['PODER'] + '-' + msc_rec['FONTE_STN']
#retirando o ponto
q = msc_rec['PODER_FONTE'].str.split(".", n=1, expand=True)
msc_rec["PODER_FTE"] = q[0]
#Filtrando só as Colunas
msc_rec = msc_rec.filter(items=['PODER_FTE', 'SI_MSC', 'SF_MSC'])

############################# RESUMINDO a Receita Realizada (do FLEX)

# Retirando o FINAL (DROP)
rec.drop(rec.tail(26).index, inplace = True)
# Excluindo os Totais
rec.dropna(inplace = True)
rec.columns = ['Poder', 'Ano_Fonte', 'FONTE_STN', 'Fonte','Desc_Fonte', 'Detalhamento', 'SI_FLEX','Debito', 'Credito', 'SF_FLEX']
# Converter uma Coluna em FLOAT 64 (número decimal)
rec['SI_FLEX'] = pd.to_numeric(rec['SI_FLEX'],errors = 'coerce')
rec['Debito'] = pd.to_numeric(rec['Debito'],errors = 'coerce')
rec['Credito'] = pd.to_numeric(rec['Credito'],errors = 'coerce')
rec['SF_FLEX'] = pd.to_numeric(rec['SF_FLEX'],errors = 'coerce')

#Somando as Colunas para criar a Fonte Completa
rec['Fonte_Completa'] = rec['Fonte'] + rec['Detalhamento']
# Criando as variáveis para substituição das Fontes 212 e 230 (só pela Fonte)
subst212 = (rec['FONTE_STN'] == '999') & (rec['Fonte'] == '212')
subst230 = (rec['FONTE_STN'] == '999') & (rec['Fonte'] == '230')
subst232 = (rec['FONTE_STN'] == '999') & (rec['Fonte'] == '232')
# Substituindo a FONTE STN 999
rec['FONTE_STN'] = np.where(subst212, '749', np.where(subst230, '501', np.where(subst232, '753', rec['FONTE_STN'])))
# Criando as variáveis para substituição da Fonte 224 (pela Fonte Completa)
subst224000000 = (rec['FONTE_STN'] == '999') & (rec['Fonte_Completa'] == '224000000')
subst224000049 = (rec['FONTE_STN'] == '999') & (rec['Fonte_Completa'] == '224000049')
subst224000050 = (rec['FONTE_STN'] == '999') & (rec['Fonte_Completa'] == '224000050')
# Substituindo a FONTE STN 999
rec['FONTE_STN'] = np.where(subst224000000, '749', np.where(subst224000049, '749', np.where(subst224000050, '749', rec['FONTE_STN'])))
    
#Deixando apenas o número do Poder
q = rec['Poder'].str.split("-", n=1, expand=True)
rec["PODER"] = q[1]
#Somando as Colunas
rec['FONTE_STN'] = rec['Ano_Fonte'] + rec['FONTE_STN']
#Somando as Colunas
rec['PODER_FTE'] = rec['PODER'] + '-' + rec['FONTE_STN']
#Filtrando só as Colunas
rec = rec.filter(items=['PODER_FTE', 'SI_FLEX', 'SF_FLEX'])
rec = rec.groupby(['PODER_FTE']).sum().reset_index()

################################################## Criando um Arquivo FINAL

# RETIRAR ESPAÇO de ambas PARA NÃO DAR ERRO NO MERGE (CUIDADO!!!!)
msc_rec = msc_rec.replace(' ',  '', regex=True)
rec = rec.replace(' ',  '', regex=True)
final_rec = msc_rec.merge(rec, on='PODER_FTE', how='outer')
#final_rec = msc_rec.merge(rec, on='PODER_FTE', how='left', suffixes=('_msc', '_rec'))
# Substituir os valores NaN por um valor específico, como zero (CUIDADO pois pode ESCONDER os NAN...ou seja, SEM DADO)
final_rec = final_rec.fillna(0)
#Fazendo a diferença
final_rec['DIF_SI'] = final_rec['SI_FLEX'] - final_rec['SI_MSC']
final_rec['DIF_SF'] = final_rec['SF_FLEX'] - final_rec['SF_MSC']

# Convertendo as colunas para tipo numérico e tratando NaNs
final_rec['DIF_SI'] = pd.to_numeric(final_rec['DIF_SI'], errors='coerce').fillna(0)
final_rec['DIF_SF'] = pd.to_numeric(final_rec['DIF_SF'], errors='coerce').fillna(0)

# Supondo que tolerancia seja um valor definido anteriormente no seu código
tolerancia = 0.01  # Exemplo de tolerância
# Sua condição ajustada usando numpy.isclose para verificar a proximidade com zero
condicao1 = ~np.isclose(final_rec['DIF_SI'], 0, atol=tolerancia)
condicao2 = ~np.isclose(final_rec['DIF_SF'], 0, atol=tolerancia)

if condicao1.any() or condicao2.any():
    resposta_final_rec = 'ERRO'
else:
    resposta_final_rec = 'OK'

final_rec_a = pd.DataFrame([resposta_final_rec], columns=['Resposta'])
# Insere novas colunas
final_rec_a.insert(0, 'Análise', 'Item 3')
final_rec_a.insert(2, 'Descrição da Análise', 'Análise da Receita Realizada por Poder e Fonte')


#########################################################  ITEM 4 - DPS EMP a LIQ  ###########################################################################################################
# Importando os Arquivos
msc_dps = msc_base.copy()

######################################### RESUMINDO A MSC ref. a DPS EMPENHADA A LIQUIDAR

#Transfomar as colunas em String
msc_dps['CONTA'] = msc_dps['CONTA'].apply(str)
msc_dps['IC1'] = msc_dps['IC1'].apply(str)
msc_dps['IC2'] = msc_dps['IC2'].apply(str)
msc_dps['IC3'] = msc_dps['IC3'].apply(str)
msc_dps['IC4'] = msc_dps['IC4'].apply(str)
msc_dps['IC5'] = msc_dps['IC5'].apply(str)
msc_dps['IC6'] = msc_dps['IC6'].apply(str)

#Pegando a Receita Realizada
msc_dps = msc_dps[msc_dps['CONTA'].str.contains("6221301", case=False, regex=True)]

#Agrupando
msc_dps = msc_dps.groupby(['IC1', 'IC3', 'TIPO_VALOR'])['VALOR'].sum().reset_index()

#Pivoteando o TIPO_VALOR para colunas
msc_dps = msc_dps.pivot_table(index=['IC1', 'IC3'],
            columns='TIPO_VALOR',
            values='VALOR').reset_index()

#Filtrando só as Colunas
msc_dps = msc_dps.filter(items=['IC1', 'IC3', 'beginning_balance', 'ending_balance'])
#Renomeando as Colunas
msc_dps.columns = ['PODER', 'FONTE_STN', 'SI_MSC', 'SF_MSC']
#Somando as Colunas
msc_dps['PODER_FONTE'] = msc_dps['PODER'] + '-' + msc_dps['FONTE_STN']
#retirando o ponto
q = msc_dps['PODER_FONTE'].str.split(".", n=1, expand=True)
msc_dps["PODER_FTE"] = q[0]
#Filtrando só as Colunas
msc_dps = msc_dps.filter(items=['PODER_FTE', 'SI_MSC', 'SF_MSC'])   

######################################### RESUMINDO a DPS EMPENHADA A LIQUIDAR (do FLEX)

# Retirando o FINAL (DROP)
dps.drop(dps.tail(26).index, inplace = True)
# Excluindo os Totais
dps.dropna(inplace = True)
# Renomeando Colunas
dps.columns = ['Poder', 'Ano_Fonte', 'FONTE_STN', 'Fonte','Desc_Fonte', 'Detalhamento', 'SI_FLEX', 'Debito', 'Credito', 'SF_FLEX']

# Converter uma Coluna em FLOAT 64 (número decimal)
dps['SI_FLEX'] = pd.to_numeric(dps['SI_FLEX'],errors = 'coerce')
dps['Debito'] = pd.to_numeric(dps['Debito'],errors = 'coerce')
dps['Credito'] = pd.to_numeric(dps['Credito'],errors = 'coerce')
dps['SF_FLEX'] = pd.to_numeric(dps['SF_FLEX'],errors = 'coerce')

#Deixando apenas o número do Poder
q = dps['Poder'].str.split("-", n=1, expand=True)
dps["PODER"] = q[1]
#Somando as Colunas
dps['FONTE_STN'] = dps['Ano_Fonte'] + dps['FONTE_STN']
#Somando as Colunas
dps['PODER_FTE'] = dps['PODER'] + '-' + dps['FONTE_STN']
#Filtrando só as Colunas
dps = dps.filter(items=['PODER_FTE', 'SI_FLEX', 'SF_FLEX'])
dps = dps.groupby(['PODER_FTE']).sum().reset_index()  

################################################## Criando um Arquivo FINAL

# RETIRAR ESPAÇO de ambas PARA NÃO DAR ERRO NO MERGE (CUIDADO!!!!)
msc_dps = msc_dps.replace(' ',  '', regex=True)
dps = dps.replace(' ',  '', regex=True)
final_dps = msc_dps.merge(dps, on='PODER_FTE', how='outer')
#final_rec = msc_rec.merge(rec, on='PODER_FTE', how='left', suffixes=('_msc', '_rec'))
# Substituir os valores NaN por um valor específico, como zero (CUIDADO pois pode ESCONDER os NAN...ou seja, SEM DADO)
final_dps = final_dps.fillna(0)
#Fazendo a diferença
final_dps['DIF_SI'] = final_dps['SI_FLEX'] - final_dps['SI_MSC']
final_dps['DIF_SF'] = final_dps['SF_FLEX'] - final_dps['SF_MSC']

# Convertendo as colunas para tipo numérico e tratando NaNs
final_dps['DIF_SI'] = pd.to_numeric(final_dps['DIF_SI'], errors='coerce').fillna(0)
final_dps['DIF_SF'] = pd.to_numeric(final_dps['DIF_SF'], errors='coerce').fillna(0)

# Supondo que tolerancia seja um valor definido anteriormente no seu código
tolerancia = 0.01  # Exemplo de tolerância
# Sua condição ajustada usando numpy.isclose para verificar a proximidade com zero
condicao1 = ~np.isclose(final_dps['DIF_SI'], 0, atol=tolerancia)
condicao2 = ~np.isclose(final_dps['DIF_SF'], 0, atol=tolerancia)

if condicao1.any() or condicao2.any():
    resposta_final_dps = 'ERRO'
else:
    resposta_final_dps = 'OK'

final_dps_a = pd.DataFrame([resposta_final_dps], columns=['Resposta'])
# Insere novas colunas
final_dps_a.insert(0, 'Análise', 'Item 4')
final_dps_a.insert(2, 'Descrição da Análise', 'Análise da Depesa Empenhada a Liquidar por Poder e Fonte')



############################################################  ITEM 5 - RP  ###############################################################################################################

# Importando os Arquivos
msc_rp = msc_base.copy()

######################################### RESUMINDO A MSC ref. o RP a PAGAR
#Transfomar as colunas em String
msc_rp['CONTA'] = msc_rp['CONTA'].apply(str)
msc_rp['IC1'] = msc_rp['IC1'].apply(str)
msc_rp['IC2'] = msc_rp['IC2'].apply(str)
msc_rp['IC3'] = msc_rp['IC3'].apply(str)
msc_rp['IC4'] = msc_rp['IC4'].apply(str)
msc_rp['IC5'] = msc_rp['IC5'].apply(str)
msc_rp['IC6'] = msc_rp['IC6'].apply(str)

# trocar o sinal de SALDOS NEGATIVOS DE RP
msc_rp['VALOR'] = msc_rp.apply(lambda x: x['VALOR'] * -1
if (x['NATUREZA_VALOR'] == 'D' and not x['TIPO_VALOR'] == 'period_change')
else x['VALOR'], axis=1)

#Pegando a Receita Realizada
msc_rp = msc_rp[msc_rp['CONTA'].str.contains("632100000", case=False, regex=True)]
#retirando o ponto
q = msc_rp['IC2'].str.split(".", n=1, expand=True)
msc_rp["Funcao_Subfuncao"] = q[0]
#retirando o ponto
q1 = msc_rp['IC3'].str.split(".", n=1, expand=True)
msc_rp["FONTE_STN"] = q1[0]
#retirando o ponto
q2 = msc_rp['IC5'].str.split(".", n=1, expand=True)
msc_rp["ND"] = q2[0]
#retirando o ponto
q3 = msc_rp['IC6'].str.split(".", n=1, expand=True)
msc_rp["AI"] = q3[0]

#Agrupando
msc_rp = msc_rp.groupby(['IC1', 'Funcao_Subfuncao', 'FONTE_STN', 'ND', 'AI', 'TIPO_VALOR'])['VALOR'].sum().reset_index()
#Pivoteando o TIPO_VALOR para colunas
msc_rp = msc_rp.pivot_table(index=['IC1', 'Funcao_Subfuncao', 'FONTE_STN', 'ND', 'AI'],
            columns='TIPO_VALOR',
            values='VALOR').reset_index()

# Extrair os 6 primeiros dígitos da coluna ND...para deixar só até o Elemento
msc_rp['ND'] = msc_rp['ND'].str.slice(0, 6)
# Removendo Coluna (Excluindo de VERDADE do dataframe >> com o ´´Inplace``)
msc_rp.drop('period_change', axis=1, inplace=True)
#Somando Tudo
msc_rp['Coluna'] = msc_rp['IC1'] + '-' + msc_rp['Funcao_Subfuncao'] + '-' + msc_rp['FONTE_STN'] + '-' + '-' + msc_rp['AI']
#Filtrando só as Colunas
msc_rp = msc_rp.filter(items=['Coluna', 'beginning_balance', 'ending_balance'])
#Renomeando as Colunas
msc_rp.columns = ['Coluna', 'SI_MSC', 'SF_MSC']
msc_rp = msc_rp.groupby(['Coluna']).sum().reset_index()

######################################### RESUMINDO o RP a PAGAR (do FLEX)
# Retirando o FINAL (DROP)
rp.drop(rp.tail(26).index, inplace = True)
# Excluindo os Totais
rp.dropna(inplace = True)
# Renomeando Colunas
rp.columns = ['Poder', 'Funcao', 'Subfuncao', 'Ano_Fonte', 'FONTE_STN', 'Fonte',
                'Desc_Fonte', 'Detalhamento', 'ND', 'AI', 'SI_FLEX',
                'Debito', 'Credito', 'SF_FLEX']
# Converter uma Coluna em FLOAT 64 (número decimal)
rp['SI_FLEX'] = pd.to_numeric(rp['SI_FLEX'],errors = 'coerce')
rp['SF_FLEX'] = pd.to_numeric(rp['SF_FLEX'],errors = 'coerce')
# A funcao para numero para retirar o 0 dos numeros de 1 a 10
rp['Funcao'] = pd.to_numeric(rp['Funcao'],errors = 'coerce')

#Deixando apenas o número do Poder
q = rp['Poder'].str.split("-", n=1, expand=True)
rp["PODER"] = q[1]
# A funcao retorna para string para somar a coluna da subfuncao
rp['Funcao'] = rp['Funcao'].apply(str)
#Somando as Colunas
rp['Funcao_Subfuncao'] = rp['Funcao'] + rp['Subfuncao']
# Extrair os 6 primeiros dígitos da coluna ND...para deixar só até o Elemento
rp['ND'] = rp['ND'].str.slice(0, 6)
#Somando as Colunas para criar a Fonte Completa
rp['Fonte_Completa'] = rp['Fonte'] + rp['Detalhamento']

# Criando as variáveis para substituição das Diversas Fontes (só pela Fonte)
subst212 = (rp['FONTE_STN'] == '999') & (rp['Fonte'] == '212')
subst230 = (rp['FONTE_STN'] == '999') & (rp['Fonte'] == '230')
subst232 = (rp['FONTE_STN'] == '999') & (rp['Fonte'] == '232')
subst233 = (rp['FONTE_STN'] == '999') & (rp['Fonte'] == '233')
subst192 = (rp['FONTE_STN'] == '999') & (rp['Fonte'] == '192')
subst196 = (rp['FONTE_STN'] == '999') & (rp['Fonte'] == '196')
# Substituindo a FONTE STN 999
rp['FONTE_STN'] = np.where(subst212, '749', 
                np.where(subst230, '501', 
                np.where(subst232, '753',
                np.where(subst233, '756',
                np.where(subst192, '711',
                np.where(subst196, '711',
                            rp['FONTE_STN']))))))

# Criando as variáveis para substituição da Fonte 215 (pela Fonte Completa)
subst215000040 = (rp['FONTE_STN'] == '999') & (rp['Fonte_Completa'] == '215000040')
subst215000060 = (rp['FONTE_STN'] == '999') & (rp['Fonte_Completa'] == '215000060')
# Substituindo a FONTE STN 999
rp['FONTE_STN'] = np.where(subst215000040, '540', np.where(subst215000060, '540', rp['FONTE_STN']))
        
# Criando as variáveis para substituição da Fonte 224 (pela Fonte Completa)
subst224000000 = (rp['FONTE_STN'] == '999') & (rp['Fonte_Completa'] == '224000000')
subst224000001 = (rp['FONTE_STN'] == '999') & (rp['Fonte_Completa'] == '224000001')
subst224000049 = (rp['FONTE_STN'] == '999') & (rp['Fonte_Completa'] == '224000049')
subst224000050 = (rp['FONTE_STN'] == '999') & (rp['Fonte_Completa'] == '224000050')
subst224000015 = (rp['FONTE_STN'] == '999') & (rp['Fonte_Completa'] == '224000015')
subst224000028 = (rp['FONTE_STN'] == '999') & (rp['Fonte_Completa'] == '224000028')
subst224000029 = (rp['FONTE_STN'] == '999') & (rp['Fonte_Completa'] == '224000029')
subst224000037 = (rp['FONTE_STN'] == '999') & (rp['Fonte_Completa'] == '224000037')
subst224000039 = (rp['FONTE_STN'] == '999') & (rp['Fonte_Completa'] == '224000039')
subst224000061 = (rp['FONTE_STN'] == '999') & (rp['Fonte_Completa'] == '224000061')
subst224000062 = (rp['FONTE_STN'] == '999') & (rp['Fonte_Completa'] == '224000062')
subst224000058 = (rp['FONTE_STN'] == '999') & (rp['Fonte_Completa'] == '224000058')
# Substituindo a FONTE STN 999
rp['FONTE_STN'] = np.where(subst224000000, '749',
                np.where(subst224000001, '552', 
                np.where(subst224000049, '749', 
                np.where(subst224000050, '749', 
                np.where(subst224000015, '569', 
                np.where(subst224000028, '660', 
                np.where(subst224000029, '660',
                np.where(subst224000037, '569', 
                np.where(subst224000039, '749', 
                np.where(subst224000061, '749', 
                np.where(subst224000062, '749',
                np.where(subst224000058, '749',
                                            rp['FONTE_STN'])))))))))))) # adicionar parenteses ao adicionar regra
#Somando as Colunas
rp['FONTE_STN'] = rp['Ano_Fonte'] + rp['FONTE_STN']
#Somando Tudo
rp['Coluna'] = rp['PODER'] + '-' + rp['Funcao_Subfuncao'] + '-' + rp['FONTE_STN'] + '-' + '-' + rp['AI']
#Filtrando só as Colunas
rp = rp.filter(items=['Coluna', 'SI_FLEX', 'SF_FLEX'])
rp = rp.groupby(['Coluna']).sum().reset_index()
    
################################################## Criando um Arquivo FINAL

# RETIRAR ESPAÇO de ambas PARA NÃO DAR ERRO NO MERGE (CUIDADO!!!!)
msc_rp = msc_rp.replace(' ',  '', regex=True)
rp = rp.replace(' ',  '', regex=True)
final_rp = msc_rp.merge(rp, on='Coluna', how='outer')

# Convertendo as colunas para tipo numérico e tratando NaNs nas colunas da MSC (pois no Flex vem dados zerados e na MSC não)
final_rp['SI_MSC'] = pd.to_numeric(final_rp['SI_MSC'], errors='coerce').fillna(0)
final_rp['SF_MSC'] = pd.to_numeric(final_rp['SF_MSC'], errors='coerce').fillna(0)

#final_rec = msc_rec.merge(rec, on='PODER_FTE', how='left', suffixes=('_msc', '_rec'))
# Tirando os valores zerados tanto da MSC como do Flex
#final_rp = final_rp.query('SI_MSC != 0 or SI_FLEX != 0')
#Fazendo a diferença
final_rp['DIF_SI'] = final_rp['SI_FLEX'] - final_rp['SI_MSC']
final_rp['DIF_SF'] = final_rp['SF_FLEX'] - final_rp['SF_MSC']
                                                    
# Supondo que tolerancia seja um valor definido anteriormente no seu código
tolerancia = 0.01  # Exemplo de tolerância
# Sua condição ajustada usando numpy.isclose para verificar a proximidade com zero
condicao1 = ~np.isclose(final_rp['DIF_SI'], 0, atol=tolerancia)
condicao2 = ~np.isclose(final_rp['DIF_SF'], 0, atol=tolerancia)

if condicao1.any() or condicao2.any():
    resposta_final_rp = 'ERRO'
else:
    resposta_final_rp = 'OK'

final_rp_a = pd.DataFrame([resposta_final_rp], columns=['Resposta'])
# Insere novas colunas
final_rp_a.insert(0, 'Análise', 'Item 5')
final_rp_a.insert(2, 'Descrição da Análise', 'Análise do RP a Pagar por Poder, Função e Subfunção, Fonte e Ano de Inscrição')


##########################################  Consolidando as Respostas  ######################################################

relacao_final = pd.concat([final_ant_a, final_a, final_rec_a, final_dps_a, final_rp_a])




